In [27]:
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import numpy as np

In [3]:
stratifiedKFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [4]:
import dataset.custom_dataset
from PIL import Image
from torchvision import transforms

TRAIN_PATH = '/opt/ml/input/data/train/data.csv'

In [5]:
df = pd.read_csv(TRAIN_PATH, index_col=0)

In [6]:



for train_idx, test_idx in stratifiedKFold.split(df['path'],df['class']):
    train_cls = [0 for _ in range(18)]
    test_cls = [0 for _ in range(18)]
    
    for i in train_idx:
        train_cls[int(df['class'].loc[i])] += 1
    
    for i in test_idx:
        test_cls[int(df['class'].loc[i])] += 1
    
    print(train_cls, test_cls)
    print(np.sum(train_cls), np.sum(test_cls))
    



[2216, 1636, 332, 2908, 3272, 436, 443, 327, 66, 582, 654, 88, 443, 328, 67, 581, 654, 87] [554, 409, 83, 727, 818, 109, 111, 82, 17, 145, 164, 21, 111, 81, 16, 146, 164, 22]
[2216, 1636, 332, 2908, 3272, 436, 444, 327, 66, 581, 655, 87, 443, 327, 66, 582, 654, 88] [554, 409, 83, 727, 818, 109, 110, 82, 17, 146, 163, 22, 111, 82, 17, 145, 164, 21]
[2216, 1636, 332, 2908, 3272, 436, 443, 327, 67, 581, 655, 87, 444, 327, 66, 582, 654, 87] [554, 409, 83, 727, 818, 109, 111, 82, 16, 146, 163, 22, 110, 82, 17, 145, 164, 22]
[2216, 1636, 332, 2908, 3272, 436, 443, 327, 67, 582, 654, 87, 443, 327, 66, 582, 655, 87] [554, 409, 83, 727, 818, 109, 111, 82, 16, 145, 164, 22, 111, 82, 17, 145, 163, 22]
[2216, 1636, 332, 2908, 3272, 436, 443, 328, 66, 582, 654, 87, 443, 327, 67, 581, 655, 87] [554, 409, 83, 727, 818, 109, 111, 81, 17, 145, 164, 22, 111, 82, 16, 146, 163, 22]


In [8]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

for train_idx, test_idx in kfold.split(df['path'], df['class']):
    train_cls = [0 for _ in range(18)]
    test_cls = [0 for _ in range(18)]
    
    for i in train_idx:
        train_cls[int(df['class'].loc[i])] += 1
    
    for i in test_idx:
        test_cls[int(df['class'].loc[i])] += 1
    
    print(train_cls, test_cls)


[2212, 1613, 338, 2913, 3278, 445, 451, 329, 65, 572, 643, 86, 446, 333, 60, 589, 664, 83] [558, 432, 77, 722, 812, 100, 103, 80, 18, 155, 175, 23, 108, 76, 23, 138, 154, 26]
[2214, 1660, 317, 2885, 3260, 446, 449, 338, 66, 588, 650, 85, 449, 328, 68, 570, 664, 83] [556, 385, 98, 750, 830, 99, 105, 71, 17, 139, 168, 24, 105, 81, 15, 157, 154, 26]
[2207, 1637, 337, 2935, 3303, 409, 449, 318, 63, 587, 640, 89, 428, 329, 70, 568, 655, 96] [563, 408, 78, 700, 787, 136, 105, 91, 20, 140, 178, 20, 126, 80, 13, 159, 163, 13]
[2219, 1648, 332, 2935, 3235, 435, 440, 327, 67, 580, 656, 85, 450, 320, 68, 595, 644, 84] [551, 397, 83, 700, 855, 110, 114, 82, 16, 147, 162, 24, 104, 89, 15, 132, 174, 25]
[2228, 1622, 336, 2872, 3284, 445, 427, 324, 71, 581, 683, 91, 443, 326, 66, 586, 645, 90] [542, 423, 79, 763, 806, 100, 127, 85, 12, 146, 135, 18, 111, 83, 17, 141, 173, 19]


In [23]:
import torch
from torch import nn
from mlp_mixer_pytorch import MLPMixer
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import f1_score

In [19]:
device = torch.device('cpu')

In [20]:
model = MLPMixer(
    image_size = 384,
    channels = 3,
    patch_size = 16,
    dim = 512,
    depth = 12,
    num_classes = 18
).to(device)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [26]:
NUM_EPOCHS = 30

transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    transforms.CenterCrop(384),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

for epoch in range(NUM_EPOCHS):

    for index, (train_idx, val_idx) in enumerate(kfold.split(df['path'],df['class'])):
        df_train = pd.DataFrame(columns=['path','class'])
        for i in train_idx:
            df_train = df_train.append({'path':df['path'].loc[i], 'class':df['class'].loc[i]}, ignore_index=True)
        
        df_val = pd.DataFrame(columns=['path','class'])
        for i in val_idx:
            df_val = df_val.append({'path':df['path'].loc[i], 'class':df['class'].loc[i]}, ignore_index=True)

        train_set = dataset.custom_dataset.Custom_Dataset(df_train, target='class', transforms=transform, train=True)
        val_set = dataset.custom_dataset.Custom_Dataset(df_val, target='class', transforms=transform, train=True)

        train_loader = DataLoader(train_set, batch_size=32, num_workers=4)
        val_loader = DataLoader(val_set, batch_size=32, num_workers=4)

        for s in ['train', 'validation']:
            if s == 'train':
                model.train()

                running_loss = 0.0
                running_acc = 0.0
                n_iter = 0
                epoch_f1 = 0.0

                for index, (images, labels) in enumerate(train_loader):
                    images = images.to(device)
                    target = labels.to(device)

                    optimizer.zero_grad()
                    logits = model(images)
                    _, preds = torch.max(logits, 1)
                    loss = criterion(logits, target)
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item() * images.size(0)
                    running_acc += torch.sum(preds == target.data)
                    epoch_f1 += f1_score(target.cpu().numpy(), preds.cpu().numpy(), average='macro')
                    n_iter += 1
                
                epoch_loss = running_loss / len(train_loader.dataset)
                epoch_acc = running_acc / len(train_loader.dataset)
                epoch_f1 = epoch_f1/n_iter

                print(f"Epoch: {epoch} -  Loss : {epoch_loss:.3f},  Accuracy : {epoch_acc:.3f},  F1-Score : {epoch_f1:.4f}")

            else:
                with torch.no_grad():
                    model.eval()

                    val_loss_items = []
                    val_acc_items = []
                    val_f1_score = 0.0
                    v_iter = 0

                    for val_batch in val_loader:
                        inputs, labels = val_batch
                        inputs = inputs.to(device)
                        labels = labels.to(device)

                        outs = model(inputs)
                        preds = torch.argmax(outs, dim=-1)

                        loss_item = criterion(outs, labels).item()
                        acc_item = (labels == preds).sum().item()

                        val_loss_items.append(loss_item)
                        val_acc_items.append(acc_item)
                        val_f1_score += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='micro')
                        v_iter += 1

                    val_loss = np.sum(val_loss_items) / len(val_loader)
                    val_acc = np.sum(val_acc_items) / len(val_set)
                    val_f1 = np.sum(val_f1_score) / v_iter
        
                print(f'val_acc : {val_acc:.3f}, val loss : {val_loss:.3f}, val f1 : {val_f1:.3f}')

KeyboardInterrupt: 